# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298619110194                   -0.85    4.0
  2   -8.300228664042       -2.79       -1.25    1.0
  3   -8.300448754113       -3.66       -1.89    4.1
  4   -8.300462659309       -4.86       -2.78    2.6
  5   -8.300464366800       -5.77       -3.09    5.9
  6   -8.300464495689       -6.89       -3.24    1.2
  7   -8.300464569829       -7.13       -3.39    5.5
  8   -8.300464610479       -7.39       -3.53    1.1
  9   -8.300464635193       -7.61       -3.72    1.0
 10   -8.300464638857       -8.44       -3.82    2.0
 11   -8.300464643114       -8.37       -4.05    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64203881870                   -0.70    5.6
  2   -16.67863401147       -1.44       -1.14    1.6
  3   -16.67922586434       -3.23       -1.88    2.4
  4   -16.67928167679       -4.25       -2.77    3.6
  5   -16.67928606468       -5.36       -3.16    5.9
  6   -16.67928621616       -6.82       -3.56    1.6
  7   -16.67928622285       -8.17       -4.05    1.1


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32512752669                   -0.56    6.8
  2   -33.33270304697       -2.12       -1.00    1.0
  3   -33.33414547242       -2.84       -1.71    3.8
  4   -33.33520979352       -2.97       -2.38    3.8
  5   -33.33692248517       -2.77       -2.58    7.9
  6   -33.33693939622       -4.77       -2.90    9.0
  7   -33.33694388646       -5.35       -3.85    2.2
  8   -33.33694389974       -7.88       -4.13    6.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298632212510                   -0.85    4.1
  2   -8.300249564000       -2.79       -1.58    1.0
  3   -8.300073282577   +   -3.75       -1.95    9.6
  4   -8.300399647321       -3.49       -2.34    1.5
  5   -8.300464313961       -4.19       -3.48    1.4
  6   -8.300464547597       -6.63       -3.83    5.0
  7   -8.300464637226       -7.05       -4.32    2.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32610231510                   -0.56    6.5
  2   -33.33056594930       -2.35       -1.27    1.0
  3   -28.56206634991   +    0.68       -0.73    8.4
  4   -33.00614108370        0.65       -1.29    8.4
  5   -33.27226517833       -0.57       -1.35    5.5
  6   -33.30709635138       -1.46       -1.77    3.8
  7   -32.56710909090   +   -0.13       -1.12    5.1
  8   -33.33413281281       -0.12       -2.27    6.0
  9   -33.33420626629       -4.13       -2.30    3.0
 10   -33.33542012565       -2.92       -2.38    1.6
 11   -33.33611158164       -3.16       -2.52    2.0
 12   -33.33661394535       -3.30       -2.73    2.1
 13   -33.33690935693       -3.53       -3.13    3.5
 14   -33.33693783502       -4.55       -3.39    2.9
 15   -33.33694078140       -5.53       -3.67    4.1
 16   -33.33694315968       -5.62       -4.02    2.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.